In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import seaborn as sns

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout
from keras.preprocessing.image import ImageDataGenerator,img_to_array
from sklearn.metrics import confusion_matrix

In [ ]:
train_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
train_df.head(10)

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(10,10))
sns.countplot(data=train_df,x='label')

In [ ]:
y_train = train_df.iloc[:,0]
y_test = test_df.iloc[:,0]

In [ ]:
y_train

In [ ]:
X_train = train_df.iloc[:,1:]
X_test = test_df.iloc[:,1:]

In [ ]:
X_train = X_train.values

In [ ]:
X_train

In [ ]:
X_test = X_test.values
X_test

In [ ]:
X_train = X_train/255
X_test = X_test/255

In [ ]:
X_train

In [ ]:
X_train = X_train.reshape([-1,28,28,1])

In [ ]:
X_train

In [ ]:
X_test = X_test.reshape([-1,28,28,1])
X_test

In [ ]:
X_train[1].shape

In [ ]:
fig,ax = plt.subplots(5,2,figsize=(20,20))

for i in range(5):
    img = X_train[i]
    ax[i,0].imshow(np.squeeze(img))
    ax[i,1].hist(img.ravel(),256,[0,1])
plt.show()

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
lb = LabelBinarizer()

In [ ]:
y_train.shape

In [ ]:
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [ ]:
y_train.shape

In [ ]:
Xtrain,Xval,ytrain,yval=train_test_split(X_train, y_train,train_size=0.75,random_state=0)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,    
        rotation_range=15,    
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=False)

In [ ]:
datagen.fit(Xtrain)

In [ ]:
model = Sequential()

In [ ]:
 model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(24, activation='softmax'))

In [ ]:
learning_rate = 1e-3
lr_decay = 1e-6
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [ ]:
history = model.fit_generator(datagen.flow(Xtrain, ytrain, batch_size=32),
                    steps_per_epoch=Xtrain.shape[0]//32,
                    epochs=10,
                    verbose=1,
                    validation_data=(Xval, yval))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Loss: " + str(score[0]))
print("Accuracy: " + str(score[1]*100) + "%")